# 匯入套件

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from tqdm import tqdm

# 資料預處理

In [ ]:
# 定義影像的轉換操作
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 將影像大小統一調整為256x256
    transforms.Grayscale(num_output_channels=1),  # 轉換為單通道灰度影像
    transforms.ToTensor(),           # 將影像轉換為PyTorch張量
    transforms.Normalize(mean=[0.485], std=[0.229]),  # 影像正規化
])

# 載入資料集

In [ ]:
# 載入訓練集
train_dataset = datasets.ImageFolder(root='chest_xray/train/', transform=transform)
# 訓練集的平均值和標準差
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 載入驗證集
val_dataset = datasets.ImageFolder(root='chest_xray/val/', transform=transform)
# 驗證集的平均值和標準差
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 載入測試集
test_dataset = datasets.ImageFolder(root='chest_xray/test/', transform=transform)
# 測試集的平均值和標準差
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 建立模型

In [ ]:
# 定義一個卷積神經網路
class CNN(nn.Module):
    def __init__(self): # 定義初始化方法
        super(CNN, self).__init__() # 繼承父類別的初始化方法
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 第一個卷積層
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 第二個卷積層
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 第三個卷積層
        self.pool = nn.MaxPool2d(2, 2)                            # 池化層
        self.adap_pool = nn.AdaptiveAvgPool2d((1, 1))             # 自適應池化層
        self.fc1 = nn.Linear(128, 512)                            # 全連接層
        self.fc2 = nn.Linear(512, 2)                              # 輸出層

    # 定義向前傳播函數，x為輸入的影像張量
    def forward(self, x):
        x = F.relu(self.conv1(x)) # 使用ReLU激活函數
        x = self.pool(x)         # 池化
        x = F.relu(self.conv2(x)) # 使用ReLU激活函數
        x = self.pool(x)        # 池化
        x = F.relu(self.conv3(x)) # 使用ReLU激活函數
        x = self.pool(x)       # 池化
        x = self.adap_pool(x)  # 自適應池化層將任何尺寸的特徵圖調整為1x1
        x = x.view(-1, 128)    # 攤平特徵圖
        x = F.relu(self.fc1(x)) # 使用ReLU激活函數
        x = self.fc2(x)        # 輸出層
        return x              # 返回輸出

# 建立一個卷積神經網路的實例
model = CNN()

# 訓練模型

In [ ]:
# 檢查是否有可用的GPU，如果沒有，則使用CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) # 印出目前使用的設備
model.to(device) # 將模型移到目前使用的設備上

# 設定損失函數和優化器
criterion = nn.CrossEntropyLoss()
# 使用Adam優化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 設定epoch數量
num_epochs = 25

# 準備記錄損失和準確率
train_losses = []  # 用於儲存每個epoch的訓練損失
val_losses = []    # 用於儲存每個epoch的驗證損失
val_accuracies = []  # 用於儲存每個epoch的驗證準確率

# 訓練模型
for epoch in range(num_epochs):
    model.train()  # 設定模型為訓練模式
    running_loss = 0.0 # 用於累積訓練損失
    train_correct = 0 # 用於累積訓練準確率
    train_total = 0  # 用於累積訓練資料的總數

    # 使用tqdm進度條展示訓練進度
    train_loop = tqdm(train_loader, position=0, leave=True) # 建立一個用於訓練的迴圈
    for images, labels in train_loop: # 迭代訓練資料集
        images, labels = images.to(device), labels.to(device)  # 將資料移到GPU上

        # 前向傳播
        outputs = model(images)
        loss = criterion(outputs, labels)  # 計算損失

        # 反向傳播和優化
        optimizer.zero_grad()
        loss.backward() # 計算梯度
        optimizer.step() # 更新權重

        running_loss += loss.item() # 累積訓練損失
        _, predicted = torch.max(outputs.data, 1) # 預測類別
        train_total += labels.size(0) # 累積訓練資料的總數
        train_correct += (predicted == labels).sum().item() # 累積訓練準確率

        # 更新進度條的顯示
        train_loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        # 顯示當前的訓練損失和準確率
        train_loop.set_postfix(loss=running_loss/train_total, accuracy=100. * train_correct / train_total)

    # 計算本輪的平均訓練損失
    train_losses.append(running_loss / len(train_loader))
    # 計算本輪的平均訓練準確率
    train_accuracy = 100. * train_correct / train_total

    # 驗證階段
    model.eval()  # 設定模型為評估模式
    val_running_loss = 0.0 # 用於累積驗證損失
    val_correct = 0 # 用於累積驗證準確率
    val_total = 0 # 用於累積驗證資料的總數
    with torch.no_grad(): # 不計算梯度
        for images, labels in val_loader: # 迭代驗證資料集
            images, labels = images.to(device), labels.to(device) # 將資料移到GPU上
            outputs = model(images) # 前向傳播
            loss = criterion(outputs, labels) # 計算損失
            val_running_loss += loss.item() # 累積驗證損失
            _, predicted = torch.max(outputs.data, 1) # 預測類別
            val_total += labels.size(0) # 累積驗證資料的總數
            val_correct += (predicted == labels).sum().item() # 累積驗證準確率

    # 計算本輪的平均驗證損失
    val_losses.append(val_running_loss / len(val_loader))
    # 計算本輪的平均驗證準確率
    val_accuracy = 100 * val_correct / val_total
    # 紀錄本輪的驗證準確率
    val_accuracies.append(val_accuracy)

    # 顯示本輪的訓練和驗證損失以及驗證準確率
    print(f'Validation Accuracy: {val_accuracy}%')

# 儲存訓練好的模型參數
torch.save(model.state_dict(), 'pneumonia_model.pth')


# 可視化訓練結果

In [ ]:
# 可視化訓練和驗證損失
plt.figure(figsize=(12, 5)) # 設定圖形大小
plt.subplot(1, 2, 1) # 建立子圖形1
plt.plot(train_losses, label='Training Loss') # 繪製訓練損失
plt.plot(val_losses, label='Validation Loss') # 繪製驗證損失
plt.title('Training and Validation Loss') # 設定圖形標題
plt.xlabel('Epoch') # 設定x軸標籤
plt.ylabel('Loss') # 設定y軸標籤
plt.legend() # 顯示圖例

# 可視化驗證準確率
plt.subplot(1, 2, 2) # 建立子圖形2
plt.plot(val_accuracies, label='Validation Accuracy') # 繪製驗證準確率
plt.title('Validation Accuracy') # 設定圖形標題
plt.xlabel('Epoch') # 設定x軸標籤
plt.ylabel('Accuracy') # 設定y軸標籤
plt.legend() # 顯示圖例

plt.show() # 顯示圖形

# 混淆矩陣、靈敏度和特異度

In [ ]:
model.eval() # 設定模型為評估模式
y_true = [] # 用於儲存真實標籤
y_pred = [] # 用於儲存預測標籤

# 不計算梯度
with torch.no_grad():
    for images, labels in test_loader: # 迭代測試資料集
        images, labels = images.to(device), labels.to(device) # 將資料移到GPU上
        outputs = model(images) # 前向傳播
        _, predicted = torch.max(outputs.data, 1) # 預測類別
        y_true.extend(labels.cpu().numpy()) # 儲存真實標籤
        y_pred.extend(predicted.cpu().numpy()) # 儲存預測標籤

# 計算混淆矩陣
conf_matrix = confusion_matrix(y_true, y_pred)

# 計算靈敏度和特異度
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn) # 計算靈敏度
specificity = tn / (tn + fp) # 計算特異度

# 可視化混淆矩陣
sns.heatmap(conf_matrix, annot=True, fmt='d') # 繪製混淆矩陣
plt.title('Confusion Matrix') # 設定圖形標題
plt.ylabel('True Label') # 設定y軸標籤
plt.xlabel('Predicted Label') # 設定x軸標籤
plt.show() # 顯示圖形

print(f'Sensitivity: {sensitivity}') # 顯示靈敏度
print(f'Specificity: {specificity}') # 顯示特異度

# AUC曲線

In [ ]:
# 模型評估模式
model.eval()

# 儲存所有標籤和模型預測的機率
all_labels = []
# 由於模型輸出的是對數機率，因此需要使用softmax函數轉換為機率
all_probs = []

# 不計算梯度
with torch.no_grad():
    for images, labels in test_loader: # 迭代測試資料集
        images, labels = images.to(device), labels.to(device) # 將資料移到GPU上
        
        # 取得模型的預測機率
        outputs = model(images)
        probabilities = torch.nn.functional.softmax(outputs, dim=1) # 使用softmax函數轉換為機率
        
        all_probs.extend(probabilities.cpu().numpy()) # 儲存預測機率
        all_labels.extend(labels.cpu().numpy()) # 儲存標籤

# 計算每個類別的機率
all_probs = np.array(all_probs)
# 由於是二分類，取第一個類別的機率
all_labels = np.array(all_labels)

# 由於是二分類，取第一個類別的機率
# 注意：如果您的類別標籤不是 {0, 1}，則需要相應調整
prob_of_pos_class = all_probs[:, 1]

# 計算 ROC 曲線
fpr, tpr, thresholds = roc_curve(all_labels, prob_of_pos_class, pos_label=1)

# 計算 AUC
roc_auc = auc(fpr, tpr)

# 繪製 ROC 曲線
plt.figure() # 建立圖形
lw = 2 # 設定線寬
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) # 繪製 ROC 曲線
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--') # 繪製對角線
plt.xlim([0.0, 1.0]) # 設定x軸範圍
plt.ylim([0.0, 1.05]) # 設定y軸範圍
plt.xlabel('False Positive Rate') # 設定x軸標籤
plt.ylabel('True Positive Rate') # 設定y軸標籤
plt.title('Receiver operating characteristic example') # 設定圖形標題
plt.legend(loc="lower right") # 顯示圖例
plt.show() # 顯示圖形


# F1 分數

In [ ]:
# 確保模型處於評估模式
model.eval()

# 儲存預測和真實標籤
all_preds = []
all_labels = []

# 無需計算梯度
with torch.no_grad():
    for images, labels in test_loader: # 迭代測試資料集
        images, labels = images.to(device), labels.to(device) # 將資料移到GPU上

        # 取得模型預測
        outputs = model(images)
        _, predicted = torch.max(outputs, 1) # 預測類別

        # 收集預測和真實標籤
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 計算分類報告
report = classification_report(all_labels, all_preds, output_dict=True)

# 轉換為 DataFrame
report_df = pd.DataFrame(report).transpose()

# 將分類報告以文字表格形式顯示
print(report_df)
